```{=latex}
\usepackage{hyperref}
\usepackage{graphicx}
\usepackage{listings}
\usepackage{textcomp}
\usepackage{fancyvrb}

\newcommand{\passthrough}[1]{\lstset{mathescape=false}#1\lstset{mathescape=true}}
\newcommand{\tightlist}{}
```

```{=latex}
\title{Unit Testing Your Web Application}
\author{Moshe Zadka -- https://cobordism.com}
\date{}

\begin{document}
\begin{titlepage}
\maketitle
\end{titlepage}

\frame{\titlepage}
```

```{=latex}
\begin{frame}
\frametitle{Acknowledgement of Country}

Belmont (in San Francisco Bay Area Peninsula)

Ancestral homeland of the Ramaytush Ohlone people

\end{frame}
```

I live in Belmont,
in the San Francisco Bay Area Peninsula.
I wish to acknowledge it as the
ancestral homeland
of the
Ramaytush Ohlone people.

In [124]:
import json
import pyramid.response
import pyramid.config
import httpx
from unittest import mock
from hamcrest import assert_that, equal_to
import io

```{=latex}
\begin{frame}
\frametitle{Outline}

Intro to Pyramid \pause

What are Unit tests? \pause

What is httpx? \pause

Unit test examples! \pause

\end{frame}
```

## Pyramid Example

```{=latex}
\begin{frame}
\frametitle{Crash Course in Pyramid}

Some quick examples!

\end{frame}
```

### JSON Renderer

```{=latex}
\begin{frame}[fragile]
\frametitle{JSON View}
```

In [165]:
def jsonv(request):
    return {}
with pyramid.config.Configurator() as config:
    config.add_route("json", "/json")
    config.add_view(jsonv, route_name="json", renderer="json")
app = config.make_wsgi_app()

```{=latex}
\end{frame}
```

In [166]:
client = httpx.Client(app=app, base_url="https://example.com/")

```{=latex}
\begin{frame}[fragile]
\frametitle{JSON View Retrieval}
```

In [167]:
request = client.get("/json")
request.json()

{}

```{=latex}
\end{frame}
```

### Using Route Matches

```{=latex}
\begin{frame}[fragile]
\frametitle{Matcher View}
```

In [168]:
def matches(request):
    return dict(name=request.matchdict["name"])
with pyramid.config.Configurator() as config:
    config.add_route("matches", "/matches/{name}")
    config.add_view(matches, route_name="matches", renderer="json")
app = config.make_wsgi_app()

```{=latex}
\end{frame}
```

In [169]:
client = httpx.Client(app=app, base_url="https://example.com/")

```{=latex}
\begin{frame}[fragile]
\frametitle{Matcher View Retrieval}
```

In [170]:
request = client.get("/matches/hello")
request.json()

{'name': 'hello'}

```{=latex}
\end{frame}
```

### Using Settings

```{=latex}
\begin{frame}[fragile]
\frametitle{Settings View}
```

In [171]:
def setting(request):
    return dict(field=request.registry.settings["field"])
with pyramid.config.Configurator(settings=dict(field="field_value")) as config:
    config.add_route("setting", "/setting")
    config.add_view(setting, route_name="setting", renderer="json")
app = config.make_wsgi_app()

```{=latex}
\end{frame}
```

In [172]:
client = httpx.Client(app=app, base_url="https://example.com/")

```{=latex}
\begin{frame}[fragile]
\frametitle{Settings View Retrieval}
```

In [173]:
request = client.get("/setting")
request.json()

{'field': 'field_value'}

```{=latex}
\end{frame}
```

### Pyramid crash course review

```{=latex}
\begin{frame}[fragile]
\frametitle{View Function}
```

In [276]:
def lookup(
    request: pyramid.request.Request
) -> str:
#    ^^^
#    Will be jsonified
    matcher = request.registry.settings["matcher"] 
#             ^^^^^^^^^^^^^^^^^^^^^^^^^
#             Access settings through request
    name = request.matchdict["name"]
#          ^^^^^^^^^^^^^^^^^
#          Get parameter from route
    return matcher.get(name.lower(), "default")
#          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
#          Business logic

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{Routing}
```

In [277]:
def include_lookup(config):
    config.add_route("lookup", "/lookup/{name}")
#          ^^^^^^^^^|^^^^^^^   ^^^^^^^^^^^^^^^
#          add a new|route    route path
#          route    |name
    config.add_view(lookup, route_name="lookup", renderer="json")
#          ^^^^^^^^|^^^^^   ^^^^^^^^^^^^^^^^^^^  ^^^^^^^^^^^^^^^
#          add a   |view    route to attach      Convert to a JSON response
#          view to |callable
#          a route |

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{Configuration}
```

In [278]:
with pyramid.config.Configurator(
#^^^ ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
#Commit at the end      ^^^^^^^^^^
#                       Configurator
#                       class
    settings=dict(matcher=dict(special="hello")),
#   ^^^^^^^^      ^^^^^^^ ^^^^^^^^^^^^^^^^^^^^^^
#   settings      settings ^^^^^^^^^^^^^^^^^^^^^
#                 key       Mapping "special"
#                           name to "hello"
) as config:
#    ^^^^^^
#    configurator
#    object
    include_lookup(config)

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{App creation}
```

In [279]:
app = config.make_wsgi_app(
#                 ^^^^
#         Web Standard Gateway Interface
)

```{=latex}
\end{frame}
```

In [280]:
client = httpx.Client(app=app, base_url="https://example.com/")
request = client.get("/lookup/SPECIAL")
request.json()

'hello'

## Unit Testing

### What is a unit test?

```{=latex}
\begin{frame}
\frametitle{Unit test: rough definition}

\pause

Runs the code

\pause

Might fail on buggy code

\pause

Self-contained

\end{frame}
```

### What is a mock?

```{=latex}
\begin{frame}
\frametitle{Mock}

\pause

Fake object

\pause

Configurable behavior

\pause

Records access

\end{frame}
```

### Goal of unit test

```{=latex}
\begin{frame}
\frametitle{Good Unit Test}

\pause

Avoid failing on valid: \pause

Public APIs \pause

Reduce assumptions
\end{frame}
```

### Patching as assumption

```{=latex}
\begin{frame}
\frametitle{Patch Makes Bad Unit Test}

\pause

Patch: Temporarily replacing a global

\pause

Assumption: Global used

\pause

Assumption: Used through path

\end{frame}
```

### Parts of unit test

```{=latex}
\begin{frame}
\frametitle{Unit test anatomy}

\pause

Set-up

\pause

Execution

\pause

Post-process \pause
(optional) \pause

Verification

\end{frame}
```

## Python Web Concepts

### WSGI

```{=latex}
\begin{frame}[fragile]
\frametitle{WSGI}

Framework \pause

to server

\end{frame}
```

### `httpx`

```{=latex}
\begin{frame}[fragile]
\frametitle{httpx}

Requests alternative \pause

Can consume WSGI directly
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{httpx with WSGI}

Creating the client: \pause
```

In [281]:
client = httpx.Client(
    base_url="https://example.com/",
    app=app,
)

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{httpx with WSGI}

Calling the client: \pause

```

In [282]:
request = client.get("/lookup/SPECIAL")

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{httpx with WSGI}

Using the httpx request object: \pause

```

In [283]:
request.json()

'hello'

```{=latex}
\end{frame}
```

## Unit test examples!

### System under test

```{=latex}
\begin{frame}[fragile]
\frametitle{System Under Test: View}
```

In [314]:
def lookup(
    request: pyramid.request.Request
) -> str:
    matcher = request.registry.settings["matcher"] 
    name = request.matchdict["name"]
    return matcher.get(name, "default")
#                      ^^^^^
#                      Bug: missing .lower()

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{System Under Test: App}
```

In [315]:
def make_app(special_value):
#^^          ^^^^^^^^^^^^^
#^^          Parameter for
#            the application
#Creating the app from a function,
#not at top level
    settings = dict(
        matcher=dict(special=special_value),
    )
    with pyramid.config.Configurator(settings=settings) as config:
        include_lookup(config)
    return config.make_wsgi_app()

```{=latex}
\end{frame}
```

### Mocking Request

```{=latex}
\begin{frame}[fragile]
\frametitle{Calling function directly}

Setup:

```

In [316]:
request = mock.MagicMock()
request.registry.settings = dict(
    matcher=dict(special="hello"),
)
request.matchdict = dict(name="SPECIAL")

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{Calling function directly}

Execute:
```

In [317]:
result = lookup(request)

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{Calling function directly}

Verification
```

In [318]:
try:
    assert_that(result, equal_to("hello"))
except AssertionError as exc:
    print(exc)


Expected: 'hello'
     but: was 'default'



```{=latex}
\end{frame}
```

### Calling WSGI

```{=latex}
\begin{frame}[fragile]
\frametitle{Calling WSGI}

Setup:
```

In [319]:
environ = dict(
    PATH_INFO="/lookup/SPECIAL",
    REQUEST_METHOD="GET",
)
start_response = mock.MagicMock(
    return_value=io.BytesIO()
)

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{Calling WSGI}

Execute:
```

In [320]:
app = make_app("hello")
base_parts = app(environ, start_response)

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{Calling WSGI}

Post-process:
```

In [321]:
parts = [start_response.return_value.getvalue()]
parts.extend(base_parts)
(status, headers), kwargs = start_response.call_args
result = json.loads(b"".join(parts).decode("utf-8"))

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{Calling WSGI}

Verification
```

In [322]:
try:
    assert_that(result, equal_to("hello"))
except AssertionError as exc:
    print(exc)


Expected: 'hello'
     but: was 'default'



```{=latex}
\end{frame}
```

### Using `httpx`

```{=latex}
\begin{frame}[fragile]
\frametitle{Using httpx}

Setup:
```

In [323]:
app = make_app("hello")
client = httpx.Client(
    base_url="https://example.com/",
    app=app,
)

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{Using httpx}

Execute:
```

In [324]:
resp = client.get("/lookup/SPECIAL")

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{Using httpx}

Post-process:
```

In [325]:
result = resp.json()

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{Using httpx}

Verification:
```

In [326]:
try:
    assert_that(result, equal_to("hello"))
except AssertionError as exc:
    print(exc)


Expected: 'hello'
     but: was 'default'



```{=latex}
\end{frame}
```

## Summary

### Pyramid: Concepts

```{=latex}
\begin{frame}
\frametitle{Summary: Pyramid}

\pause

View: \pause
request, response, route \pause

Route: \pause
URL match \pause

Registry: \pause
Parameters, shared objects \pause

Configurator: \pause
Views, Routes, Registry \pause

WSGI: \pause
Configurator to server \pause
(or httpx!)

\end{frame}
```

### Unit Testing

```{=latex}
\begin{frame}
\frametitle{Summary: Unit Test}

\pause

Verify code, \pause
quickly, \pause
and safely: \pause

Mock! \pause

Pre-plan \pause

Prefer stable APIs

\end{frame}
```

## Bonus material

### Pyramid Concepts

#### View

```{=latex}
\begin{frame}
\frametitle{View}

Function \pause

Argument: Request \pause

Return value: Response

\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{View Route}

Route \pause

to View \pause

\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{View Route Predicates}

Route to view can be conditional:\pause

Request type,\pause

Content type,\pause

Arbitrary predicates

\end{frame}
```

#### Route

```{=latex}
\begin{frame}
\frametitle{Route}

Path \pause

to route name \pause

\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{Route: advanced}

\pause

Path fragment matching \pause

...and more

\end{frame}
```

#### Registry

```{=latex}
\begin{frame}
\frametitle{Registry: Application Parameters}

\pause

Settings: ad-hoc dictionary

\end{frame}
```

#### Configurator

```{=latex}
\begin{frame}[fragile]
\frametitle{Configurator}

Routes \pause

Views \pause

Registry \pause

\end{frame}
```

### Routing extras

#### Static view

```{=latex}
\begin{frame}[fragile]
\frametitle{Static Value}
```

In [310]:
def empty(request):
    return pyramid.response.Response(
        json.dumps({}).encode("ascii"),
        content_type="application/json",
    )
with pyramid.config.Configurator() as config:
    config.add_route("root", "/")
    config.add_view(empty, route_name="root")
app = config.make_wsgi_app()

```{=latex}
\end{frame}
```

In [311]:
client = httpx.Client(app=app, base_url="https://example.com/")

```{=latex}
\begin{frame}[fragile]
\frametitle{Static View Retrieval}
```

In [312]:
request = client.get("/")
request.json()

{}

```{=latex}
\end{frame}
```

#### Using Parameters

```{=latex}
\begin{frame}[fragile]
\frametitle{Parameter View}
```

In [313]:
def params(request):
    return dict(thing=request.params["thing"])
with pyramid.config.Configurator() as config:
    config.add_route("params", "/params")
    config.add_view(params, route_name="params", renderer="json")
app = config.make_wsgi_app()

```{=latex}
\end{frame}
```

In [299]:
client = httpx.Client(app=app, base_url="https://example.com/")

```{=latex}
\begin{frame}[fragile]
\frametitle{Parameter View Retrieval}
```

In [300]:
request = client.get("/params?thing=hello")
request.json()

{'thing': 'hello'}

```{=latex}
\end{frame}
```

#### Using Request Body

```{=latex}
\begin{frame}[fragile]
\frametitle{Body View}
```

In [301]:
def body(request):
    return dict(field=request.json_body["field"])
with pyramid.config.Configurator() as config:
    config.add_route("body", "/body")
    config.add_view(body, route_name="body", renderer="json")
app = config.make_wsgi_app()

```{=latex}
\end{frame}
```

In [302]:
client = httpx.Client(app=app, base_url="https://example.com/")

```{=latex}
\begin{frame}[fragile]
\frametitle{Body View Retrieval}
```

In [303]:
request = client.post("/body", json=dict(field="hello"))
request.json()

{'field': 'hello'}

```{=latex}
\end{frame}
```

### Registry extras

```{=latex}
\begin{frame}
\frametitle{Registry: Application Parameters}

\pause

Utility: Scalable configuration


\end{frame}
```

In [304]:
import zope.interface
import attrs

```{=latex}
\begin{frame}[fragile]
\frametitle{Interfaces and Components}
```

In [305]:
class IValueGetter(zope.interface.Interface):
    def get_value(self) -> int:
        ...

@zope.interface.implementer(IValueGetter)
@attrs.frozen
class ValueStorer:
    _value: int
    
    def get_value(self):
        return self._value

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{Interface-based Registry}
```

In [306]:
config = pyramid.config.Configurator()
config.registry.registerUtility(ValueStorer(value=42))
config.registry.getUtility(IValueGetter).get_value()

42

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{Why Interface-based Registry?}

Namespaced\pause

Semantically meaningful \pause

Configuration

\end{frame}
```

### Configurator extras

```{=latex}
\begin{frame}[fragile]
\frametitle{Configurator Inclusion}
```

In [307]:
def root_stuff(config):
    config.add_route("root", "/")
    config.add_view(empty, route_name="root")    
with pyramid.config.Configurator() as config:
    config.include(root_stuff)
    app = config.make_wsgi_app()

In [308]:
client = httpx.Client(app=app, base_url="https://example.com/")

In [309]:
request = client.get("/")
request.json()

{}

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{Configurator Inclusion}

Include allows: \pause

\begin{itemize}
\item Callable \pause
\item Callable dotted name \pause
\item Module (will use "includeme") \pause
\item Module dotted name
\end{itemize}

\end{frame}
```

### Scanning

```{=latex}
\begin{frame}[fragile]
\frametitle{Configurator Scanning}

* Scan a package
* Find functions decorated "view config"

\end{frame}
```

```{=latex}
\end{document}
```